In [2]:
import parmed as pmd
import subprocess as sp
import numpy as np

from openeye.oechem import *
from openeye.oeomega import *
from openforcefield.typing.engines.smirnoff import *
from pdbfixer import PDBFixer

In [3]:
ben_smiles = '[O-]C(=O)c1ccccc1'
bcd_smiles = 'C([C@@H]1[C@@H]2[C@@H]([C@H]([C@H](O1)O[C@@H]3[C@H](O[C@@H]([C@@H]([C@H]3O)O)O[C@@H]4[C@H](O[C@@H]([C@@H]([C@H]4O)O)O[C@@H]5[C@H](O[C@@H]([C@@H]([C@H]5O)O)O[C@@H]6[C@H](O[C@@H]([C@@H]([C@H]6O)O)O[C@@H]7[C@H](O[C@@H]([C@@H]([C@H]7O)O)O[C@@H]8[C@H](O[C@H](O2)[C@@H]([C@H]8O)O)CO)CO)CO)CO)CO)CO)O)O)O'

In [5]:
mols = []
for smiles in [ben_smiles, bcd_smiles]:
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, smiles)
    for atom in mol.GetAtoms():
        atom.SetPartialCharge(atom.GetFormalCharge())
    oechem.OEAddExplicitHydrogens(mol)
    oechem.OETriposAtomNames(mol)
    mols.append(mol)

In [7]:
omegaOpts = OEOmegaOptions()
omega = OEOmega(omegaOpts)

for mol in mols:
    if omega(mol):
        oequacpac.OEAssignCharges(mol, oequacpac.OEAM1BCCELF10Charges())
        conf = mol.GetConf(oechem.OEHasConfIdx(0))
        absFCharge = 0
        sumFCharge = 0
        sumPCharge = 0.0
        for atm in mol.GetAtoms():
            sumFCharge += atm.GetFormalCharge()
            absFCharge += abs(atm.GetFormalCharge())
            sumPCharge += atm.GetPartialCharge()
        print("{}: {} formal charges give total charge {}"
              "; sum of partial charges {:5.4f}".format(mol.GetTitle(), absFCharge,
                                                        sumFCharge, sumPCharge))
        # oechem.OEWriteMolecule(ofs, conf)


: 1 formal charges give total charge -1; sum of partial charges -1.0000
: 0 formal charges give total charge 0; sum of partial charges 0.0000


In [14]:
for atom in mols[0].GetAtoms():
        print(atom, atom.GetPartialCharge())

 0 C -0.16098999977111816
 1 C -0.1662600040435791
 2 C -0.1662600040435791
 3 C -0.10735999792814255
 4 C -0.10735999792814255
 5 C -0.12452999502420425
 6 C 0.9059900045394897
 7 O -0.8342499732971191
 8 O -0.8342499732971191
 9 H 0.09599000215530396
10 H 0.1014999970793724
11 H 0.1014999970793724
12 H 0.14813999831676483
13 H 0.14813999831676483


In [16]:
ofs = oemolostream()
ofs.open('ben-openeye.mol2')
OEWriteMolecule(ofs, mols[0])

0

In [21]:
omega.GetMaxConfs()

200

In [22]:
ofs = oemolostream()
ofs.open('bcd-openeye.mol2')
OEWriteMolecule(ofs, mols[1])

0

In [23]:
mols[1]

<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7f8eb709fe40> >

In [25]:
mols[1].NumConfs()

13

Why are there 13 conformations?

# Map between atoms!

In [4]:

import sys
import string

import lxml.etree as etree

from simtk.openmm.app import element as elem
from simtk.openmm.app import Topology
from openforcefield.utils import generateTopologyFromOEMol, get_data_filename

import os
import math
import copy
import re
import numpy
import random

import openeye.oechem
import openeye.oeomega
import openeye.oequacpac

from openeye import oechem, oequacpac

from openforcefield.typing.chemistry import ChemicalEnvironment, SMIRKSParsingError

from simtk import openmm, unit

import time

import networkx as nx

import itertools


In [5]:
def generateGraphFromTopology(topology):
    """Geneate a NetworkX graph from a Topology object.
    Parameters
    ----------
    topology : simtk.openmm.app.Topology
        The source topology.
    Returns
    -------
    graph : networkx.Graph
        The resulting graph, with nodes labeled with atom indices and elements
    """
    import networkx as nx
    # Create graph of atoms connected by bonds.
    G = nx.Graph()
    for atom in topology.atoms():
        G.add_node(atom.index, element=atom.element)
    for (atom1, atom2) in topology.bonds():
        G.add_edge(atom1.index, atom2.index)

    return G

In [6]:
bcd_smiles = 'C([C@@H]1[C@@H]2[C@@H]([C@H]([C@H](O1)O[C@@H]3[C@H](O[C@@H]([C@@H]([C@H]3O)O)O[C@@H]4[C@H](O[C@@H]([C@@H]([C@H]4O)O)O[C@@H]5[C@H](O[C@@H]([C@@H]([C@H]5O)O)O[C@@H]6[C@H](O[C@@H]([C@@H]([C@H]6O)O)O[C@@H]7[C@H](O[C@@H]([C@@H]([C@H]7O)O)O[C@@H]8[C@H](O[C@H](O2)[C@@H]([C@H]8O)O)CO)CO)CO)CO)CO)CO)O)O)O'
bcd_smiles_mol = oechem.OEMol()
oechem.OESmilesToMol(bcd_smiles_mol, bcd_smiles)
oechem.OEAddExplicitHydrogens(bcd_smiles_mol)

True

In [7]:
bcd_smiles_mol.NumAtoms()

147

In [8]:
# oenb.draw_mol(bcd_smiles_mol)

In [9]:
# ofs = oemolostream()
# ofs.open('tmp.mol2')
# ofs.SetFormat(OEFormat_MOL2)
# oechem.OEWriteMolecule(ofs, bcd_smiles_mol)
# ofs.close()

In [10]:
bcd_gaff = 'bcd-ben/bcd-sybyl.mol2'
ifs = oechem.oemolistream(bcd_gaff)
for mol in ifs.GetOEGraphMols():
    bcd_gaff_mol = oechem.OEMol(mol)
ifs.close()

In [11]:
bcd_gaff_mol.NumAtoms()

147

In [12]:
bcd_smiles_topology = generateTopologyFromOEMol(bcd_smiles_mol)

In [13]:
bcd_gaff_topology = generateTopologyFromOEMol(bcd_gaff_mol)

Exception: Error: Reference molecule must have unique atom names in order to create a Topology.

In [14]:
for atom in bcd_gaff_mol.GetAtoms():
    print(atom.GetName())

C1
H1
O1
C2
H2
O2
HO2
C3
H3
O3
HO3
C4
H4
C5
H5
O5
C6
H61
H62
O6
HO6
C1
H1
O1
C2
H2
O2
HO2
C3
H3
O3
HO3
C4
H4
C5
H5
O5
C6
H61
H62
O6
HO6
C1
H1
O1
C2
H2
O2
HO2
C3
H3
O3
HO3
C4
H4
C5
H5
O5
C6
H61
H62
O6
HO6
C1
H1
O1
C2
H2
O2
HO2
C3
H3
O3
HO3
C4
H4
C5
H5
O5
C6
H61
H62
O6
HO6
C1
H1
O1
C2
H2
O2
HO2
C3
H3
O3
HO3
C4
H4
C5
H5
O5
C6
H61
H62
O6
HO6
C1
H1
O1
C2
H2
O2
HO2
C3
H3
O3
HO3
C4
H4
C5
H5
O5
C6
H61
H62
O6
HO6
C1
H1
O1
C2
H2
O2
HO2
C3
H3
O3
HO3
C4
H4
C5
H5
O5
C6
H61
H62
O6
HO6


In [15]:
OETriposAtomNames(bcd_gaff_mol)

In [16]:
for atom in bcd_gaff_mol.GetAtoms():
    print(atom.GetName())

C1
H1
O1
C2
H2
O2
H3
C3
H4
O3
H5
C4
H6
C5
H7
O4
C6
H8
H9
O5
H10
C7
H11
O6
C8
H12
O7
H13
C9
H14
O8
H15
C10
H16
C11
H17
O9
C12
H18
H19
O10
H20
C13
H21
O11
C14
H22
O12
H23
C15
H24
O13
H25
C16
H26
C17
H27
O14
C18
H28
H29
O15
H30
C19
H31
O16
C20
H32
O17
H33
C21
H34
O18
H35
C22
H36
C23
H37
O19
C24
H38
H39
O20
H40
C25
H41
O21
C26
H42
O22
H43
C27
H44
O23
H45
C28
H46
C29
H47
O24
C30
H48
H49
O25
H50
C31
H51
O26
C32
H52
O27
H53
C33
H54
O28
H55
C34
H56
C35
H57
O29
C36
H58
H59
O30
H60
C37
H61
O31
C38
H62
O32
H63
C39
H64
O33
H65
C40
H66
C41
H67
O34
C42
H68
H69
O35
H70


In [17]:
bcd_gaff_topology = generateTopologyFromOEMol(bcd_gaff_mol)

```python
        self._reference_to_topology_atom_mappings = { reference_molecule : list() for reference_molecule in self._reference_molecules }
        for molecule_graph in nx.connected_component_subgraphs(G):
            # Check if we have already stored a reference molecule for this molecule.
            reference_molecule_exists = False
            for (reference_molecule_graph, reference_molecule) in zip(self._reference_molecule_graphs, self._reference_molecules):
                GM = isomorphism.GraphMatcher(molecule_graph, reference_molecule_graph)
                if GM.is_isomorphic():
                    # This molecule is present in the list of unique reference molecules.
                    reference_molecule_exists = True
                    # Add the reference atom mappings.
                    reference_to_topology_atom_mapping = dict()
                    for (topology_atom, reference_atom) in GM.mapping.items():
                        reference_to_topology_atom_mapping[reference_atom] = topology_atom
                    self._reference_to_topology_atom_mappings[reference_molecule].append(reference_to_topology_atom_mapping)
                    # Break out of the search loop.
                    break
```

In [18]:
G1 = generateGraphFromTopology(bcd_smiles_topology)
G2 = generateGraphFromTopology(bcd_gaff_topology)

molecule_graph = G1
reference_molecule_graph = G2

reference_to_topology_atom_mappings = list()


from networkx.algorithms import isomorphism

GM = isomorphism.GraphMatcher(molecule_graph, reference_molecule_graph)
if GM.is_isomorphic():
    reference_to_topology_atom_mapping = dict()
    for (topology_atom, reference_atom) in GM.mapping.items():
        reference_to_topology_atom_mapping[reference_atom] = topology_atom
    reference_to_topology_atom_mappings.append(reference_to_topology_atom_mapping)

In [19]:
reference_to_topology_atom_mapping

{0: 9,
 1: 10,
 2: 12,
 3: 7,
 4: 8,
 5: 109,
 6: 144,
 7: 5,
 8: 6,
 9: 110,
 10: 145,
 11: 3,
 12: 4,
 13: 1,
 14: 2,
 15: 11,
 16: 0,
 17: 112,
 18: 113,
 19: 111,
 20: 146,
 21: 18,
 22: 19,
 23: 26,
 24: 20,
 25: 21,
 26: 25,
 27: 115,
 28: 22,
 29: 23,
 30: 24,
 31: 114,
 32: 13,
 33: 14,
 34: 15,
 35: 16,
 36: 17,
 37: 107,
 38: 141,
 39: 142,
 40: 108,
 41: 143,
 42: 32,
 43: 33,
 44: 40,
 45: 34,
 46: 35,
 47: 39,
 48: 117,
 49: 36,
 50: 37,
 51: 38,
 52: 116,
 53: 27,
 54: 28,
 55: 29,
 56: 30,
 57: 31,
 58: 105,
 59: 138,
 60: 139,
 61: 106,
 62: 140,
 63: 46,
 64: 47,
 65: 54,
 66: 48,
 67: 49,
 68: 53,
 69: 119,
 70: 50,
 71: 51,
 72: 52,
 73: 118,
 74: 41,
 75: 42,
 76: 43,
 77: 44,
 78: 45,
 79: 103,
 80: 135,
 81: 136,
 82: 104,
 83: 137,
 84: 60,
 85: 61,
 86: 68,
 87: 62,
 88: 63,
 89: 67,
 90: 121,
 91: 64,
 92: 65,
 93: 66,
 94: 120,
 95: 55,
 96: 56,
 97: 57,
 98: 58,
 99: 59,
 100: 101,
 101: 132,
 102: 133,
 103: 102,
 104: 134,
 105: 74,
 106: 75,
 107: 82,
 108

In [20]:
for (reference_atom, target_atom) in reference_to_topology_atom_mapping.items():
    print(reference_atom, target_atom)

0 9
1 10
2 12
3 7
4 8
5 109
6 144
7 5
8 6
9 110
10 145
11 3
12 4
13 1
14 2
15 11
16 0
17 112
18 113
19 111
20 146
32 13
28 22
24 20
21 18
22 19
23 26
25 21
26 25
27 115
29 23
30 24
31 114
33 14
34 15
35 16
36 17
37 107
38 141
39 142
40 108
41 143
53 27
49 36
45 34
42 32
43 33
44 40
46 35
47 39
48 117
50 37
51 38
52 116
54 28
55 29
56 30
57 31
58 105
59 138
60 139
61 106
62 140
74 41
70 50
66 48
63 46
64 47
65 54
67 49
68 53
69 119
71 51
72 52
73 118
75 42
76 43
77 44
78 45
79 103
80 135
81 136
82 104
83 137
95 55
91 64
87 62
84 60
85 61
86 68
88 63
89 67
90 121
92 65
93 66
94 120
96 56
97 57
98 58
99 59
100 101
101 132
102 133
103 102
104 134
116 69
112 78
108 76
105 74
106 75
107 82
109 77
110 81
111 123
113 79
114 80
115 122
117 70
118 71
119 72
120 73
121 99
122 129
123 130
124 100
125 131
128 90
126 88
127 89
129 91
130 92
131 96
132 125
133 93
134 94
135 95
136 124
137 83
138 84
139 85
140 86
141 87
142 97
143 126
144 127
145 98
146 128


In [21]:
for atom in bcd_gaff_mol.GetAtoms():
    idx = atom.GetIdx()
    syb = OEGetAtomicSymbol(atom.GetAtomicNum())
    charge = atom.GetPartialCharge()
    print(idx, syb, charge)

0 C 0.30452901124954224
1 H 0.09078200161457062
2 O -0.4530700147151947
3 C 0.09219899773597717
4 H 0.10292600095272064
5 O -0.5866389870643616
6 H 0.4250650107860565
7 C 0.09321100264787674
8 H 0.06446799635887146
9 O -0.5925670266151428
10 H 0.42101699113845825
11 C 0.12762099504470825
12 H 0.08066099882125854
13 C 0.1043429970741272
14 H 0.06851600110530853
15 O -0.4273830056190491
16 C 0.14108100533485413
17 H 0.05738399922847748
18 H 0.05738399922847748
19 O -0.5935549736022949
20 H 0.42202699184417725
21 C 0.30452901124954224
22 H 0.09078200161457062
23 O -0.4530700147151947
24 C 0.09219899773597717
25 H 0.10292600095272064
26 O -0.5866389870643616
27 H 0.4250650107860565
28 C 0.09321100264787674
29 H 0.06446799635887146
30 O -0.5925670266151428
31 H 0.42101699113845825
32 C 0.12762099504470825
33 H 0.08066099882125854
34 C 0.1043429970741272
35 H 0.06851600110530853
36 O -0.4273830056190491
37 C 0.14108100533485413
38 H 0.05738399922847748
39 H 0.05738399922847748
40 O -0.593554

In [22]:
for atom in bcd_smiles_mol.GetAtoms():
    idx = atom.GetIdx()
    syb = OEGetAtomicSymbol(atom.GetAtomicNum())
    charge = atom.GetPartialCharge()
    print(idx, syb, charge)

0 C 0.0
1 C 0.0
2 H 0.0
3 C 0.0
4 H 0.0
5 C 0.0
6 H 0.0
7 C 0.0
8 H 0.0
9 C 0.0
10 H 0.0
11 O 0.0
12 O 0.0
13 C 0.0
14 H 0.0
15 C 0.0
16 H 0.0
17 O 0.0
18 C 0.0
19 H 0.0
20 C 0.0
21 H 0.0
22 C 0.0
23 H 0.0
24 O 0.0
25 O 0.0
26 O 0.0
27 C 0.0
28 H 0.0
29 C 0.0
30 H 0.0
31 O 0.0
32 C 0.0
33 H 0.0
34 C 0.0
35 H 0.0
36 C 0.0
37 H 0.0
38 O 0.0
39 O 0.0
40 O 0.0
41 C 0.0
42 H 0.0
43 C 0.0
44 H 0.0
45 O 0.0
46 C 0.0
47 H 0.0
48 C 0.0
49 H 0.0
50 C 0.0
51 H 0.0
52 O 0.0
53 O 0.0
54 O 0.0
55 C 0.0
56 H 0.0
57 C 0.0
58 H 0.0
59 O 0.0
60 C 0.0
61 H 0.0
62 C 0.0
63 H 0.0
64 C 0.0
65 H 0.0
66 O 0.0
67 O 0.0
68 O 0.0
69 C 0.0
70 H 0.0
71 C 0.0
72 H 0.0
73 O 0.0
74 C 0.0
75 H 0.0
76 C 0.0
77 H 0.0
78 C 0.0
79 H 0.0
80 O 0.0
81 O 0.0
82 O 0.0
83 C 0.0
84 H 0.0
85 C 0.0
86 H 0.0
87 O 0.0
88 C 0.0
89 H 0.0
90 O 0.0
91 C 0.0
92 H 0.0
93 C 0.0
94 H 0.0
95 O 0.0
96 O 0.0
97 C 0.0
98 O 0.0
99 C 0.0
100 O 0.0
101 C 0.0
102 O 0.0
103 C 0.0
104 O 0.0
105 C 0.0
106 O 0.0
107 C 0.0
108 O 0.0
109 O 0.0
110 O 0.0


In [23]:
for (reference_atom, target_atom) in reference_to_topology_atom_mapping.items():
    print(bcd_gaff_mol.GetAtom(OEHasAtomIdx(reference_atom)), 
          bcd_smiles_mol.GetAtom(OEHasAtomIdx(target_atom)))

 0 C  9 C
 1 H 10 H
 2 O 12 O
 3 C  7 C
 4 H  8 H
 5 O 109 O
 6 H 144 H
 7 C  5 C
 8 H  6 H
 9 O 110 O
10 H 145 H
11 C  3 C
12 H  4 H
13 C  1 C
14 H  2 H
15 O 11 O
16 C  0 C
17 H 112 H
18 H 113 H
19 O 111 O
20 H 146 H
32 C 13 C
28 C 22 C
24 C 20 C
21 C 18 C
22 H 19 H
23 O 26 O
25 H 21 H
26 O 25 O
27 H 115 H
29 H 23 H
30 O 24 O
31 H 114 H
33 H 14 H
34 C 15 C
35 H 16 H
36 O 17 O
37 C 107 C
38 H 141 H
39 H 142 H
40 O 108 O
41 H 143 H
53 C 27 C
49 C 36 C
45 C 34 C
42 C 32 C
43 H 33 H
44 O 40 O
46 H 35 H
47 O 39 O
48 H 117 H
50 H 37 H
51 O 38 O
52 H 116 H
54 H 28 H
55 C 29 C
56 H 30 H
57 O 31 O
58 C 105 C
59 H 138 H
60 H 139 H
61 O 106 O
62 H 140 H
74 C 41 C
70 C 50 C
66 C 48 C
63 C 46 C
64 H 47 H
65 O 54 O
67 H 49 H
68 O 53 O
69 H 119 H
71 H 51 H
72 O 52 O
73 H 118 H
75 H 42 H
76 C 43 C
77 H 44 H
78 O 45 O
79 C 103 C
80 H 135 H
81 H 136 H
82 O 104 O
83 H 137 H
95 C 55 C
91 C 64 C
87 C 62 C
84 C 60 C
85 H 61 H
86 O 68 O
88 H 63 H
89 O 67 O
90 H 121 H
92 H 65 H
93 O 66 O
94 H 120 H
96 H 56 H

In [24]:
for (reference_atom, target_atom) in reference_to_topology_atom_mapping.items():
    reference = bcd_gaff_mol.GetAtom(OEHasAtomIdx(reference_atom))
    reference_idx = reference.GetIdx()
    reference_syb = OEGetAtomicSymbol(reference.GetAtomicNum())
    reference_charge = reference.GetPartialCharge()

    template = bcd_smiles_mol.GetAtom(OEHasAtomIdx(target_atom))
    template_idx = template.GetIdx()
    template_syb = OEGetAtomicSymbol(template.GetAtomicNum())
    template_charge = template.GetPartialCharge()

    print(reference_idx, reference_syb, reference_charge, '\t',
         template_idx, template_syb, template_charge)

0 C 0.30452901124954224 	 9 C 0.0
1 H 0.09078200161457062 	 10 H 0.0
2 O -0.4530700147151947 	 12 O 0.0
3 C 0.09219899773597717 	 7 C 0.0
4 H 0.10292600095272064 	 8 H 0.0
5 O -0.5866389870643616 	 109 O 0.0
6 H 0.4250650107860565 	 144 H 0.0
7 C 0.09321100264787674 	 5 C 0.0
8 H 0.06446799635887146 	 6 H 0.0
9 O -0.5925670266151428 	 110 O 0.0
10 H 0.42101699113845825 	 145 H 0.0
11 C 0.12762099504470825 	 3 C 0.0
12 H 0.08066099882125854 	 4 H 0.0
13 C 0.1043429970741272 	 1 C 0.0
14 H 0.06851600110530853 	 2 H 0.0
15 O -0.4273830056190491 	 11 O 0.0
16 C 0.14108100533485413 	 0 C 0.0
17 H 0.05738399922847748 	 112 H 0.0
18 H 0.05738399922847748 	 113 H 0.0
19 O -0.5935549736022949 	 111 O 0.0
20 H 0.42202699184417725 	 146 H 0.0
32 C 0.12762099504470825 	 13 C 0.0
28 C 0.09321100264787674 	 22 C 0.0
24 C 0.09219899773597717 	 20 C 0.0
21 C 0.30452901124954224 	 18 C 0.0
22 H 0.09078200161457062 	 19 H 0.0
23 O -0.4530700147151947 	 26 O 0.0
25 H 0.10292600095272064 	 21 H 0.0
26 O -

In [25]:
for (reference_atom, target_atom) in reference_to_topology_atom_mapping.items():
    reference = bcd_gaff_mol.GetAtom(OEHasAtomIdx(reference_atom))
    reference_idx = reference.GetIdx()
    reference_syb = OEGetAtomicSymbol(reference.GetAtomicNum())
    reference_charge = reference.GetPartialCharge()

    template = bcd_smiles_mol.GetAtom(OEHasAtomIdx(target_atom))
    template_idx = template.GetIdx()
    template_syb = OEGetAtomicSymbol(template.GetAtomicNum())
    # template_charge = template.GetPartialCharge()
    
    template.SetPartialCharge(reference_charge)
    template_charge = template.GetPartialCharge()

    print(reference_idx, reference_syb, reference_charge, '\t',
         template_idx, template_syb, template_charge)

0 C 0.30452901124954224 	 9 C 0.30452901124954224
1 H 0.09078200161457062 	 10 H 0.09078200161457062
2 O -0.4530700147151947 	 12 O -0.4530700147151947
3 C 0.09219899773597717 	 7 C 0.09219899773597717
4 H 0.10292600095272064 	 8 H 0.10292600095272064
5 O -0.5866389870643616 	 109 O -0.5866389870643616
6 H 0.4250650107860565 	 144 H 0.4250650107860565
7 C 0.09321100264787674 	 5 C 0.09321100264787674
8 H 0.06446799635887146 	 6 H 0.06446799635887146
9 O -0.5925670266151428 	 110 O -0.5925670266151428
10 H 0.42101699113845825 	 145 H 0.42101699113845825
11 C 0.12762099504470825 	 3 C 0.12762099504470825
12 H 0.08066099882125854 	 4 H 0.08066099882125854
13 C 0.1043429970741272 	 1 C 0.1043429970741272
14 H 0.06851600110530853 	 2 H 0.06851600110530853
15 O -0.4273830056190491 	 11 O -0.4273830056190491
16 C 0.14108100533485413 	 0 C 0.14108100533485413
17 H 0.05738399922847748 	 112 H 0.05738399922847748
18 H 0.05738399922847748 	 113 H 0.05738399922847748
19 O -0.5935549736022949 	 111

In [26]:
template

<openeye.oechem.OEAtomBase; proxy of <Swig Object of type 'OEChem::OEAtomBase *' at 0x7f889a7d3e40> >

In [29]:
ofs = oemolostream()
ofs.open('bcd-openeye-antechamber-charges.mol2')
OEWriteMolecule(ofs, bcd_smiles_mol)

0

# Maybe it doesn't show up because it's 2D and I need to generate a conformation?

In [31]:
from openeye.oeomega import *

In [32]:
omegaOpts = OEOmegaOptions()
omegaOpts.SetMaxConfs(1)
omega = OEOmega(omegaOpts)

In [33]:
omega(bcd_smiles_mol)

True

In [34]:
ofs = oemolostream()
ofs.open('bcd-openeye-antechamber-charges.mol2')
OEWriteMolecule(ofs, bcd_smiles_mol)

0